# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
import math
import random

In [ ]:
# Load data from Excel file
data = pd.read_excel('Lorenz Dataset.xlsx', header=None).values  
# data = pd.read_excel('Tehran Stock Exchange.xlsx', header=None).values

data_size = len(data) 

In [ ]:
# Normalize the input data
for ii in range(len(data[0])): 
    data[:, ii] = data[:, ii] / np.max(data[:, ii])  

num_feature = len(data[0]) - 1 

In [ ]:
for row in data:
    for elem in row:
        if abs(elem) > 1:
            print(row, elem) 

In [ ]:
#Lorenz Dataset
# np.random.seed(42)
# epochs = 100 
# regularization_strength = 0.003
# learning_rate = 0.001 

#Stock Dataset
np.random.seed(41)
epochs = 100 
regularization_strength = 0.008
learning_rate = 0.0001

In [ ]:
class GMDHNeuron:
    def __init__(self, x1_index, x2_index):
        self.x1_index = x1_index
        self.x2_index = x2_index
        self.weights = np.random.uniform(-1, 1, 6) 
        self.train_mse_history = []
        self.validation_mse = None
        
    def train(self, x1_train, x2_train, actual_output, epochs,learning_rate, regularization_strength):
        self.train_mse_history = []
        for epoch in range(epochs):
            for i in range(len(x1_train)):
                predicted_output = self.predict(x1_train[i], x2_train[i])
                error = predicted_output - actual_output[i]
#                 print('index: ', i)
#                 print('error: ', error)
#                 print('x1_train[i]**2: ',x1_train[i]**2)
#                 print('x2_train[i]**2: ',x2_train[i]**2)
                
                gradient = np.array([
                    -1 * error * x1_train[i]**2,
                    -1 * error * x2_train[i]**2,
                    -1 * error * x1_train[i] * x2_train[i],
                    -1 * error * x1_train[i],
                    -1 * error * x2_train[i],
                    -1 * error
                ])
                
                # L2 regularization
                regularization_term = 2 * regularization_strength * self.weights  
                
                self.weights = self.weights - learning_rate * gradient - regularization_term
            
            train_mse = self.calculate_mse(x1_train, x2_train, actual_output)
#             print(train_mse)
            self.train_mse_history.append(train_mse)
        
        # calculate validation MSE
        self.validation_mse = self.calculate_mse(validation_data[:,self.x1_index],validation_data[:,self.x2_index],validation_output)
        
    def predict(self, x1, x2):
        return self.weights[0] * x1**2 + self.weights[1] * x2**2 + self.weights[2] * x1 * x2 + self.weights[3] * x1  + self.weights[4] * x2 + self.weights[5]

    def calculate_mse(self, x1, x2, target):
        predicted_output = self.predict(x1, x2)
        mse = np.mean((predicted_output - target) ** 2)
        return mse


    

In [ ]:
# Split data into training, validation, and testing sets
train_size = int(0.7 * data_size)
validation_size = int(0.15 * data_size)

train_data, validation_data, test_data = data[:train_size,:num_feature], data[train_size:train_size + validation_size,:num_feature], data[train_size + validation_size:,:num_feature]
train_output, validation_output, test_output = data[:train_size,num_feature], data[train_size:train_size + validation_size,num_feature], data[train_size + validation_size:,num_feature]

In [ ]:
gmdh_neurons = []
for i in range(num_feature):
    for j in range(i + 1, num_feature):
        neuron = GMDHNeuron(i,j)
        neuron.train(train_data[:, i], train_data[:, j], train_output, epochs,learning_rate, regularization_strength)
        gmdh_neurons.append(neuron)

In [ ]:
for neuron in gmdh_neurons:
    # Plotting the training data and output
    plt.figure(figsize=(20, 8)) 
    plt.semilogy(np.arange(1, epochs + 1 ), neuron.train_mse_history)
    plt.xlabel('Epoch')
    plt.ylabel('MSE Train') 
    plt.show()
    print('selected_neurons-indexs: ', neuron.x1_index,neuron.x2_index)
    print("Final Weights:", neuron.weights)
    print("Final Train MSE:", neuron.train_mse_history[-1])
    print("Final Validation MSE:", neuron.validation_mse)
    

In [ ]:
sorted_neurons = sorted(gmdh_neurons, key=lambda x: x.validation_mse)
# Select the top two neurons with the lowest validation_mse
best_neurons = sorted_neurons[:2]
print("Two smallest validation MSE values:", best_neurons)

In [ ]:
best_neurons_output = []
for neuron in best_neurons:  
    print('indexes: ', neuron.x1_index,neuron.x2_index)
    # calculate output
    output = neuron.predict(train_data[:,neuron.x1_index],train_data[:,neuron.x2_index])
    best_neurons_output.append(output)

In [ ]:
inputs = np.array(list(zip(*best_neurons_output))) 
# print(inputs)

In [ ]:
# Define the RBF function
def radial_basis_function(x, c, sigma):
    return np.exp(-0.5 * np.linalg.norm(x - c) ** 2 / (sigma ** 2))

# Initialize parameters 
num_input_neurons = 2
num_rbf_neurons = 12
learning_rate = 0.15

# Generate random initial values for weights, cluster centers, and standard deviations
np.random.seed(42)
w =  2 * np.random.rand(num_rbf_neurons) - 1
centers = np.random.rand(num_rbf_neurons, num_input_neurons)
sigmas = np.random.rand(num_rbf_neurons)

# Training data (example)
X_train = inputs
y_train = train_output

epochs = 100  # Number of training iterations
error_data_train = np.zeros(len(X_train))
mse_train = np.zeros(epochs)
# Training loop
for epoch in range(epochs):
    total_error = 0
    output_data_train = np.zeros(len(X_train))
    for i in range(len(X_train)): 
        # Forward pass
        x = X_train[i]
        rbfs_output = np.array([radial_basis_function(x, c, sigma) for c, sigma in zip(centers, sigmas)])
        y = np.dot(rbfs_output, w)
        output_data_train[i] = y
        # Calculate error
        error = y_train[i] - y
        error_data_train[i] = error
        total_error += 0.5 * (error ** 2)

        # Backpropagation
        for j in range(num_rbf_neurons):
            # Update weights
            delta_w = -error * rbfs_output[j]
            w[j] -= learning_rate * delta_w

            # Update cluster centers
            delta_c = (error * w[j] * (x - centers[j])) / (sigmas[j] ** 2)
            centers[j] -= learning_rate * delta_c

            # Update standard deviations
            delta_sigma = (error * w[j] * np.linalg.norm(x - centers[j]) ** 2) / (sigmas[j] ** 3)
            sigmas[j] -= learning_rate * delta_sigma
    mse_train[epoch] = np.mean(error_data_train ** 2)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Error = {total_error}")
    
# # Plotting the training data and output
    plt.figure(figsize=(20, 8))
    plt.subplot(2, 2, 1)
    plt.plot(y_train[5:])
    plt.plot(output_data_train[5:], 'r', linewidth=0.5)
    plt.xlabel('Train Data')
    plt.ylabel('Output')
    plt.legend(['Actual', 'Predicted']) 
    plt.tight_layout()
    plt.show()
    
    # Plotting the training MSE
    plt.subplot(2, 2, 2)
    plt.semilogy(np.arange(1, epoch + 1), mse_train[:epoch])
    plt.xlabel('Epoch')
    plt.ylabel('MSE Train') 
    
    print('Epoch: {} \t'.format(epoch+1))
    print('total_error: ',total_error)
    print('MSE_train: {:.4f}'.format(mse_train[epoch]))

In [ ]:
-0.01556572 -0.01568061
total_error:  0.3582647650886961
MSE_train: 0.0005